In [1]:
# POWER MODEL MAIN
# Created: 10/06/2022
# Author: Cameron King
# Function Authors: Henri Wessels, Alec Church

# Description:
#
# This is the main script that models the power collection, transmission, and receiving systems/processes.
# Generally takes in satellite design choices and varies altitude and transmission time to see what orbits/transmission
# times are possible with current choices, and which of those are most efficient. 
# These outputs of most efficient range of orbit/time, as well as sub-models made during the formation of this model, 
# will be given to other groups within the team to inform on their design decisions as well as ROI, mass, MOI, etc.


# libraries
import math
import numpy
import mpmath

# functions
from ipynb.fs.full.efficiency_funcs import *
from ipynb.fs.full.transmission_funcs import *
from ipynb.fs.full.GaussNCone import *

In [2]:
# Step 1:
#
# power_collection_func (Henri), outputs Power available

# Inputs: Solar panel area, panel efficiency, storage efficiency, battery choices, IMPORTANT: Comms & other sat power usage

# no looping, just determines the power available for transmission given choices above. 
# Will need to be scaleable, to be able to vary total collected power (incase transmission losses are calculated to be too big, and we need more collected)

# outputs: P_C (Power collected), P_A (Power Available for transmission)


In [6]:
# Step 2:
#
# determines values for power_transmission_func
# defines constants and core variables h & t

#############################################################
######### THESE ARE THE ONLY THINGS YOU SHOULD VARY #########
#############################################################

# Laser Wattage

P_C = 10000000 # assumed to be 10 MW now, this will come from Henri's power budgeting
               # Note that this isn't power transmitted in kWh, that depends on transmission time and therefore orbit


# Variables 
h = 50000; # Passover Altitude, m
r = 10; # m, radius of the reciever
r_aperture = 0.25; # m, radius of output lens on sc

# errors, change to greater than radius of receiver to see how efficiency rapidly drops
pos_err = [0,0,0];
point_err = [0,0];

##################################################################
##################################################################
##################################################################

# Constants
mu_m = 4.905E12; # Gravitational parameter of the moon, 
r_m = 1737500; # radius of the moon, m

# calculate transmission time for transmitting if in view
d_max = math.sqrt(2*r_m*h + h**2)
x_max = math.sqrt(d_max**2 - h**2)
V = math.sqrt(mu_m/(r_m+h)); # orbital velocity

t_end = 2*x_max/V; # Transmisson time, s


# time step accuracy
t_step = t_end/1000; # size of time step, s
N = t_end/t_step; # number of elements in every model array ALWAYS USE THIS FOR LOOPS
N = int(N)

# Functions #
def FOV_func(x,r,h): # determines the angular field of view of receiver, as func of x, r, and h
    FOV = 2*(numpy.arctan((x+r)/h)-numpy.arctan(x/h))
    return FOV


# Calculations
x_max = V*t_end/2; # Maximum transmission ground distance, x(t=0) at start of transmission
FOV_min = FOV_func(x_max,r,h); # smallest field of view, at maximum distance

# ideal beam radius: the average radius of receiver over transmission period
r_min = math.sqrt(x_max**2+h**2)*numpy.tan(FOV_min/2); # minimum radius based on FOV
r_b = (r + r_min)/2; # average visual radius, ideal beam radius at surface

# average transmission distance
d_max = numpy.sqrt(x_max**2+h**2);
d_ave = (d_max + h)/2;


# Outputs of d_ave and r_b are attained for Step 3


In [7]:
# Step 3:
#
# power_transmission_func (Alec)
# Loop through power transmission function, determine flux dispersion, power transmitted, and power losses
# ASSUMES WARM UP TIME NEGLIGABLE


# inputs from above: r_b, d_ave (conditions for beam focal distance and radius at focal point)

# preallocationss:

t = numpy.zeros(N);
x = numpy.zeros(N);
d = numpy.zeros(N);
F_disp = [];
P_T = [];
                     
for i in range(0,N-1):
    t[i] = i*t_step;
    x[i] = V*t[i] - x_max;
    
    # For d = d_max -> h -> d_max, N elements
    d[i] = math.sqrt(x[i]**2 + h**2)
    
    current_disp = gaussNcone_transmission_func(r_aperture,r_b,d_ave,d[i],P_C)
    current_disp = numpy.array(current_disp)
    F_disp.append(current_disp) 
    P_T.append(current_disp[1,0])

F_disp = numpy.array(F_disp)

# outputs for every d:   r70, r90, r99 @ surface (if there are better %s then go for it, this is just an example)

# outputs (for transmission time assumed) constants: P_T (total power transmitted, W), P_hl (Power loss due to heating of laser)
# IMPORTANT: this needs to be the total power transmitted IN WATTS, not energy (kWh) because then power_transmission_func 
# is not dependent on transmission time, and power integration is easier down the line



In [8]:
### Step 4:
#
# Determining efficiencies and power received
# loops through the transmission period to determine actual amount of power received
# accounts for pointing and position errors, as well as losses due to incident angle and efficiency of the reciever

# preallocation
t = numpy.zeros(N);
x = numpy.zeros(N);
theta = numpy.zeros(N);
n_rec = numpy.zeros(N);
n_pos = numpy.zeros(N);
E_R = numpy.zeros(N);
E_T = numpy.zeros(N);

for i in range(0,N-1):
    t[i] = i*t_step;
    x[i] = V*t[i] - x_max;
    theta[i] = mpmath.atan(x[i]/h);
        
    n_rec[i] = receiver_eff_func(theta[i], P_T[i]);
    # this function calculates the efficiency associated with incidence angle and receiver efficiency
    # it will be between 0 & 1, where 1 is 100% efficient and 0 is 0%.
    # This efficiency includes the flux dispursion in the analysis
    n_pos[i] = position_eff_func(theta[i], pos_err, point_err, F_disp[i,:,:], h, r);
    # this function will determine the efficiiency associated with the pointing and position error of the satellite
    # it will be between 0 & 1, where 1 is 100% efficient and 0 is 0%.
    # effiency = F_hit/F_beam (areas normalized to flux)
    # each 'shell' (r70,r90,r99) will include a certain amount of W, and the % of that shell that hits the receiver is the efficiency of that shell

    E_R[i] = t_step*P_T[i]*n_rec[i]*n_pos[i]
    E_T[i] = t_step*P_T[i]
    #print(n_pos[i])
        
E_R_tot = sum(E_R); # This is in Joules ->
E_R_tot = E_R_tot*2.77*10**-7 # kWh

E_T_tot = sum(E_T); # This is in Joules ->
E_T_tot = E_T_tot*2.77*10**-7 # kWh

print('Total Transmission time (time in view) is', round(t_end,2),'s')
print('Energy received is',round(E_R_tot,3),'kWh');
print('Energy Transmitted is',round(E_T_tot,3),'kWh');
print('Energy Efficiency',round(E_R_tot/E_T_tot*100,2),'%');





Total Transmission time (time in view) is 503.26 s
Energy received is 275.16 kWh
Energy Transmitted is 1391.253 kWh
Energy Efficiency 19.78 %


In [ ]:
# Step 5:
#
# Monte-Carlo of the whole situation:
# vary h & t, keep theta, theta_dot below certian values

# find optimal orbit altitude and transmission time values, as well as sizing for design variables to hit objectives
